In [3]:
import pandas as pd
import numpy as np
import os

# Path from which the file is taken for cleanup
os.chdir('C:/Users/nkumar25/Documents/3-10-2025 apparel wave 1 file creation')

# Read the CSV file
df_i= pd.read_csv("Wave 1.csv", encoding='ISO-8859-1')
df_i.drop_duplicates()
df_v = pd.read_csv('Visibility.csv', encoding='ISO-8859-1')
df_v.drop_duplicates()
df_m = pd.read_csv("multi.csv", encoding='ISO-8859-1')
df_m.drop_duplicates()
print('Reading and duplication removal complete')

#creating a blank list to hold multiselect attribute values
at_code_list = list()
for index, row in df_m.iterrows():
    if row['MULTI_SELECTION_ELIGIBILTY_IND']=='Y' and row['DEFAULT_ATTR_DATA_TYPE_CODE']!='SEL':
        at_code_list.append(row['ATTR_CODE'])

# Create the 'con' column for both DataFrames
df_i['con'] = df_i['DIVISION'].astype(str)+df_i['DEPT'].astype(str)+df_i['PROD_TYPE_HIER_CODE']+df_i['SUB_PROD_TYPE_HIER_CODE']
df_v['con'] = df_v['DIVISION'].astype(str)+df_v['DEPT'].astype(str)+df_v['PROD_TYPE_HIER_CODE']+df_v['SUB_PROD_TYPE_HIER_CODE']

# Ensure unique keys before merging
df_v.drop_duplicates(subset=['con'], inplace=True)

# Merge the DataFrames on the 'con' column
df = df_i.merge(df_v, on='con', how="left")
print('merged visibility table and removed non Y')
# remove blAnks that are NUM and TXT from secondary value
df = df[~(df['ATTR_CODE_x'].isin(at_code_list) & df['ATTR_VALUE_SECONDARY'].isnull())]
print('removed blanks from multiselect list')

# Function to round values if conditions are met
def round_if_number(row):
    if row['ATTR_CODE_x'] in at_code_list and pd.notnull(row['ATTR_VALUE_SECONDARY']):
        try:
            row['ATTR_VALUE_SECONDARY'] = round(float(row['ATTR_VALUE_SECONDARY']))
        except ValueError:
            pass
    return row
print('secondary value rounding complete')

# Apply the function to each row
df = df.apply(round_if_number, axis=1)

atc_to_check = [
    '1ST_COMP_FBR_CONTENT', '2ND_COMP_FBR_CONTENT', '2ND_ITEM_FILLING_CONTENT',
    '3RD_COMP_FBR_CONTENT', '4TH_COMP_FBR_CONTENT', 'BASE_MATERIAL', 'BATH_RUG_MATERIAL',
    'BED_SKIRT_FIBER_CONTENT', 'BEDDING_FIBER_CONTENT', 'BEDDING_FILLING_CONTENT',
    'CASE_MATERIALS', 'CURTAIN_PANEL_FIBER_CONTENT', 'DECORATIVE_PILLOW_FIBER_CONTENT',
    'DECORATIVE_PILLOW_FILLING_CONTENT', 'FBR_CNTNT_BACK_PANEL', 'FBR_CNTNT_CUP_OTSHLL',
    'FBR_CNTNT_FRONT_PANL', 'FBR_CNTNT_SIDE_PANEL', 'FIBER_CONTENT', 'FIBER_CONTENT_BACK',
    'FIBER_CONTENT_TOP', 'FIBER_CONTENT-BRIEF', 'FIBER_CONTENT-CROTCH', 'FIBER_CONTENT-LEGS',
    'FILLING_CONTENT', 'FRAME_CONTENT', 'HAND_TOWEL_BASE_MATERIAL', 'OUTER_SURFACE_CONSTR',
    'SHAM_FIBER_CONTENT', 'SHEETS_FIBER_CONTENT', 'SHOWER_CURTAIN_HOOKS_MATERIAL',
    'SHOWER_CURTAIN/LINER_MATERIAL', 'SOLE_OUTER_SURF_AREA', 'TIE_BACK_FIBER_CONTENT',
    'UPHOLSTERY_CONTENT', 'UPPR_OUTER_SURF_AREA', 'VALANCE_FIBER_CONTENT'
]

loc_val_code = df.columns.get_loc("ATTR_VALUE_CODE")
loc_val_sec = df.columns.get_loc("ATTR_VALUE_SECONDARY")

attr_code_df = pd.DataFrame(atc_to_check)
list_diff = []
o_list = []
log = pd.DataFrame()



at_index = df.index[df['ATTR_CODE_x'] == 'RFID']
for i in range(len(at_index)):
    if df.iloc[at_index[i], loc_val_code] == 'NO' or pd.isnull(df.iloc[at_index[i], loc_val_code]):
        df.iloc[at_index[i], loc_val_code] = 'NO'
        df.iloc[at_index[i], loc_val_sec] = None
#for index, row in df.iterrows():
   # if row['']
print('RFID cleanup complete')
for index, row in attr_code_df.iterrows():
    attr_code = row[0]
    t = df[df['ATTR_CODE_x'] == attr_code]
    tsort = t.sort_values(by='ITEM')
    unique_orins = tsort['ITEM'].unique()
    uni_df = pd.DataFrame(unique_orins)

    for index, row in uni_df.iterrows():
        orin = row[0]
        orin_df = tsort[tsort['ITEM'] == orin]
        orin_df['ATTR_VALUE_SECONDARY'] = pd.to_numeric(orin_df['ATTR_VALUE_SECONDARY'], errors='coerce')
        sum_avs = orin_df['ATTR_VALUE_SECONDARY'].sum()
        sum_avs = int(sum_avs)
        dif = 100 - sum_avs

        if dif > 0 and dif != 100:
            loc = df.index[(df['ITEM'] == orin) & (df['ATTR_CODE_x'] == attr_code)].to_list()
            if loc:
                lr = loc[0]
                copy_row = df.iloc[lr].copy()
                copy_row['ATTR_VALUE_CODE'] = 'NOT_APPLICABLE'
                copy_row['ATTR_VALUE_SECONDARY'] = dif
                df = pd.concat([df, pd.DataFrame([copy_row])]).reset_index(drop=True)
                log = pd.concat([log, pd.DataFrame([copy_row])])
        elif dif < 0:
            list_diff.append(dif)
            o_list.append(orin)

psyc_df = pd.DataFrame({'Diff': list_diff, 'orin': o_list})
print('Equal to 100 check complete')

file_out = 'C:/Users/nkumar25/Documents/3-10-2025 apparel wave 1 file creation/Output'
log_out = 'C:/Users/nkumar25/Documents/3-10-2025 apparel wave 1 file creation/Log'
prob_out = 'C:/Users/nkumar25/Documents/3-10-2025 apparel wave 1 file creation/Prob'

out_path = os.path.join(file_out, "Creation.csv")
log_path = os.path.join(log_out, "log.csv")
prob_path = os.path.join(prob_out, "Problems6.csv")

df.to_csv(out_path, index=False)
log.to_csv(log_path, index=False)
psyc_df.to_csv(prob_path, index=False)

Reading and duplication removal complete
merged visibility table and removed non Y
removed blanks from multiselect list
secondary value rounding complete
RFID cleanup complete


C:\Users\nkumar25\AppData\Local\Temp\ipykernel_13048\49887436.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orin_df['ATTR_VALUE_SECONDARY'] = pd.to_numeric(orin_df['ATTR_VALUE_SECONDARY'], errors='coerce')


Equal to 100 check complete
